#Importo librerie

In [1]:
import pandas as pd
import numpy as np
import concurrent.futures
import time
from requests.exceptions import ReadTimeout

In [2]:
!pip install -U -q PyDrive

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Autenticazione Spotify API

In [ ]:
!pip uninstall spotipy

In [ ]:
!pip install spotipy

In [4]:
# autenticazione Spotify API con spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
auth_manager = SpotifyClientCredentials(client_id='caf57b996b464996bff50ab59186f265', client_secret='0bfcdbff8015426cae855b56b692f69b')
sp = spotipy.Spotify(auth_manager=auth_manager, requests_timeout=10)

#Importo dataset

In [5]:
# autenticazione google drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
# importo seconda parte del dataset --> 7087 datapoints

drive.CreateFile({'id':'1YAAvPUaPeBIddkuVzWgw7fhPPcT2uJTY'}).GetContentFile('billboard_dataset_unique.csv')
df_billboard = pd.read_csv("billboard_dataset_unique.csv").drop('Unnamed: 0',axis=1).iloc[7087:14174]

drive.CreateFile({'id':'1eOqgPk_izGXKIT5y6KfqPkmKWqBonVc0'}).GetContentFile('dataset2_X_billboard.csv')
df_songs = pd.read_csv("dataset2_X_billboard.csv").drop('Unnamed: 0',axis=1)


# df_billboard.iloc[:7087]

# df_billboard.iloc[7087:14174]

# df_billboard.iloc[14174:21261]

# df_billboard.iloc[21261:]

In [7]:
df_billboard.head()

,title,artist,weeks
7087,Trying To Find Atlantis,Jamie O'Neal,4
7088,Piggy Bank,50 Cent,1
7089,Why Do You Love Me,Garbage,1
7090,The Widow,The Mars Volta,2
7091,"Awful, Beautiful Life",Darryl Worley,20


In [8]:
df_billboard.shape

(7087, 3)

#Definizione funzioni

In [9]:
def print_exec_time(start):
  print("Esecuzione completata in %.4f secondi" % (time.perf_counter()-start))

In [10]:
# funzione che effettua ricerca con Spotify API considerando i casi in cui nel campo 'artist' siano presenti più artisti (featuring)

def search_fix(artist, title):
  artist_separators = ['%%%', ' Featuring', ' featuring', ' feat.', ' Feat.', ' feat', ' Feat', ' &', ' x', ' X', ' with', ' With', ', ', '/', ' duet', ' Duet', '+', ' and']
  title_separators = ['%%%', ' (']
  title_fix = ["%%%", "'s", "'"]

  id = None

  for x in artist_separators:
    for y in title_separators:
      for z in title_fix:
        try:
          id = sp.search(q='artist:'+artist.split(x)[0]+' track:'+title.split(y)[0].replace(z, ''), type='track', limit=1)['tracks']['items'][0]['id']
        except IndexError:
          pass
        if(id != None):
          break
      if(id != None):
        break
    if(id != None):
      break

  return id

In [11]:
# funzione che prendendo una singola riga del Billboard dataset restituisce una lista con id, artista e titolo
# --> in caso di errore l'id viene impostato a None

def get_id(row):

  artist = row[1]
  title = row[0]

  print("fetching id for %s by %s ..." % (title, artist))

  try:
    try:
      id = sp.search(q='artist:'+artist+' track:'+title, type='track', limit=1)['tracks']['items'][0]['id']
    except IndexError:
      id = search_fix(artist, title)
  except ReadTimeout:
    id = None
      
  if(id == None):
    print('--> [error] %s by %s' % (title, artist))

  return [id, artist, title]

In [12]:
# funzione che, preso un id, restituisce un array con le features (audio e non) della traccia corrispondente

def get_features(id):
  print("fetching features for id: %s" % id)

  # audio features
  danceability = []
  energy = []
  key = []
  loudness =[]
  mode = []
  speechiness = []
  acousticness = []
  instrumentalness = []
  liveness = []
  valence = []
  tempo = []
  duration_ms = []

  audio_features_array = [danceability, energy, key, loudness, mode, speechiness,
      acousticness, instrumentalness, liveness, valence, tempo, duration_ms]

  # altre features
  release_date = []
  explicit = []

  release_date.append(sp.track(id)['album']['release_date'])
  explicit.append(sp.track(id)['explicit'])

  audio_features = sp.audio_features(id)[0]

  try:
    # rimuovo campi non necessari
    to_remove = ['type', 'id', 'uri', 'track_href', 'analysis_url', 'time_signature']
    for rmv in to_remove:
        audio_features.pop(rmv)

    for i, feature in enumerate(audio_features.keys()):
        audio_features_array[i].append(audio_features[feature])

  except AttributeError:
    print("--> [error] id = %s" % id)

    for i in range(12):
        audio_features_array[i].append(None)


  audio_features_array.append(release_date)
  audio_features_array.append(explicit) 

  return audio_features_array

#Integrazione dataset

##Recupero gli id del Billboard dataset

In [13]:
time_0 = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor() as executor:
  results = executor.map(get_id, df_billboard.values.tolist())
  
  output = []
  for result in results:
    output.append(result)

print_exec_time(time_0)

Output streaming troncato alle ultime 5000 righe.
fetching id for Nowhere To Go by Melissa Etheridge ...
fetching id for You Can Make History (Young Again) by Elton John ...
fetching id for All I Want by Susanna Hoffs ...
fetching id for Another You, Another Me by Brady Seals ...
fetching id for When Boy Meets Girl by Total ...
fetching id for Macarena (Bayside Boys Mix) by Los Del Rio ...
fetching id for Key West Intermezzo (I Saw You First) by John Mellencamp ...
fetching id for Bow Down by Westside Connection ...
fetching id for Bohemian Rhapsody (From "High School High") by The Braids ...
fetching id for Birmingham by Amanda Marshall ...
fetching id for At Night I Pray by Wild Orchid ...
fetching id for Dance Into The Light by Phil Collins ...
fetching id for One And One by Robert Miles Featuring Maria Nayler ...
fetching id for Tell Me (From "Eddie") by Dru Hill ...
--> [error] Sugar Honey Ice Tea by Goodfellaz
fetching id for Let's Make A Night To Remember by Bryan Adams ...
fetc

In [14]:
# creo backup del billboard dataset
df_billboard_bak = df_billboard.copy()

In [15]:
# inserisco gli id ottenuti in una nuova colonna nel df_billboard
ids = np.array(output)[:,0]
df_billboard.insert(0, 'id', ids)

In [16]:
# calcolo percentuale di canzoni trovate
found_id = df_billboard.id.count()
x = (found_id / df_billboard.title.count()) * 100
print("Found ids = %d%%" % x)

Found ids = 90%


In [17]:
# esporto su google drive
from google.colab import drive

# mounts the google drive to Colab Notebook
drive.mount('/content/drive',force_remount=True)

df_billboard.to_csv('/content/drive/My Drive/Colab Notebooks/datasets/billboard+ids_1.csv')

Mounted at /content/drive


##Recupero audio features del Billboard dataset

In [18]:
# reimporto dataset billboard (con ids) + dataset principale
"""
drive.CreateFile({'id':'1fZzuYu-HXKP9HUeio-FL9P4eNygOQ0qq'}).GetContentFile('billboard+ids_0.csv') 
df_billboard = pd.read_csv("billboard+ids_0.csv").drop('Unnamed: 0',axis=1)

drive.CreateFile({'id':'1eOqgPk_izGXKIT5y6KfqPkmKWqBonVc0'}).GetContentFile('dataset2_X_billboard.csv')
df_songs = pd.read_csv("dataset2_X_billboard.csv").drop('Unnamed: 0',axis=1)
"""

'\ndrive.CreateFile({\'id\':\'1fZzuYu-HXKP9HUeio-FL9P4eNygOQ0qq\'}).GetContentFile(\'billboard+ids_0.csv\') \ndf_billboard = pd.read_csv("billboard+ids_0.csv").drop(\'Unnamed: 0\',axis=1)\n\ndrive.CreateFile({\'id\':\'1eOqgPk_izGXKIT5y6KfqPkmKWqBonVc0\'}).GetContentFile(\'dataset2_X_billboard.csv\')\ndf_songs = pd.read_csv("dataset2_X_billboard.csv").drop(\'Unnamed: 0\',axis=1)\n'

In [19]:
# elimino valori nulli (= id non trovati)
df_billboard = df_billboard.dropna()

# creo lista con gli id del dataset billboard
ids = list(df_billboard.id.array)

In [20]:
# creo lista degli id che non sono presenti nel dataset principale
time_0 = time.perf_counter()
ids_new = [id for id in ids if id not in list(df_songs.id.array)]
print_exec_time(time_0)

Esecuzione completata in 1484.5960 secondi


In [21]:
time_0 = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor() as executor:
  results = executor.map(get_features, ids_new)

  output = []
  for result in results:
    output.append(result)

print_exec_time(time_0)

fetching features for id: 1sC3kJPO47Y6xJC2o1BY7u
fetching features for id: 29z2e1GVOwxTvW0ZTOyQxQ
fetching features for id: 5meJWQuHyHDsOjWF8AtSDe
fetching features for id: 0yftAXSqtZoHdgrXjSuZwz
fetching features for id: 7wWvmpIDhNt4RZKWKKxvcu
fetching features for id: 7GgeZtT72GXe9rfJVL5NUt
fetching features for id: 3UHjwsergHnehp7o4XKdzm
fetching features for id: 6GVryHJpjnrnrsTggoNTpe
fetching features for id: 4lxreqqyX8FBYgwClz3TRa
fetching features for id: 46a3BjDPBdi6OngVhTkuM6
fetching features for id: 6YvKQtwW8YgIqRDbb4dFDK
fetching features for id: 57ZXcBtCZXSg9TVV5xRdnR
fetching features for id: 5TV96lkS2NF6kzILjORzLs
fetching features for id: 6HFXjB2fvQUBuE3B5kmvfW
fetching features for id: 38j7IKSmSfAWLeRNNZPXNV
fetching features for id: 78LHQroYp0bq3PZD4f4bpw
fetching features for id: 7D7XFT6clDMVDP6q3QM16V
fetching features for id: 0jSIovJtKxEXH6vI4oRx7I
fetching features for id: 5q8WL51d8vXvGSwDdfzgJ0
fetching features for id: 7HGTkn2aX7MNdKs7nV2xBt
fetching features fo

#Inserisco audio features nel dataset Billboard

In [22]:
num_datapoints = np.array(output).shape[0]

output = np.array(output).reshape((num_datapoints,14))

In [23]:
# creo backup df_billboard
df_billboard_bak = df_billboard.copy()

# filtro dataset billboard tenendo solo id nell'array 'ids_new', quindi quelli che non sono presenti nel dataset principale
df_billboard = df_billboard[df_billboard.id.isin(ids_new)]

In [24]:
to_insert = ['danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',  
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'duration_ms',
    'release_date',
    'explicit']

for i, col in enumerate(output.T):
    df_billboard.insert(4, to_insert[i], col)

In [25]:
# converto colonna 'release_date' in tipo datetime
df_billboard.release_date = pd.to_datetime(df_billboard.release_date,format="%Y-%m-%d",exact=False)

In [26]:
# inserisco colonna 'year'
year = df_billboard['release_date'].apply(lambda x: int(x.year))
df_billboard.insert(6, 'year', year)

In [27]:
# inserisco colonna 'popularity' --> nb: inizializzo a 0 perchè verrà rimossa
df_billboard.insert(17, 'popularity', np.zeros(df_billboard.shape[0]))

In [28]:
# inserisco colonna 'hit'
hit = np.ones(df_billboard.shape[0])
df_billboard.insert(3, 'hit', hit)
df_billboard.hit = df_billboard.hit.apply(int)

In [29]:
df_billboard.head()

,id,title,artist,hit,weeks,explicit,release_date,year,duration_ms,tempo,valence,liveness,instrumentalness,acousticness,speechiness,mode,loudness,key,popularity,energy,danceability
7087,1sC3kJPO47Y6xJC2o1BY7u,Trying To Find Atlantis,Jamie O'Neal,1,4,False,2005-01-01,2005,208827,167.983,0.787,0.29,4.7e-06,0.156,0.0444,1,-4.355,4,0.0,0.883,0.61
7089,29z2e1GVOwxTvW0ZTOyQxQ,Why Do You Love Me,Garbage,1,1,True,2005-04-01,2005,234227,160.038,0.468,0.33,1.85e-05,0.00052,0.0378,1,-2.43,7,0.0,0.877,0.469
7090,5meJWQuHyHDsOjWF8AtSDe,The Widow,The Mars Volta,1,2,False,2005-02-10,2005,350880,106.222,0.106,0.276,0.00927,0.628,0.0349,0,-6.059,9,0.0,0.658,0.236
7095,0yftAXSqtZoHdgrXjSuZwz,Monday Morning Church,Alan Jackson,1,16,False,2004-08-24,2004,204467,135.922,0.154,0.135,0,0.226,0.0321,1,-15.062,4,0.0,0.26,0.557
7098,7wWvmpIDhNt4RZKWKKxvcu,Nothin 'Bout Love Makes Sense,LeAnn Rimes,1,19,False,2015-02-03,2015,177307,121.933,0.796,0.13,0,0.566,0.0369,1,-3.244,9,0.0,0.802,0.656


#Esporto

In [30]:
# esporto in google drive
from google.colab import  drive

# mounts the google drive to Colab Notebook
drive.mount('/content/drive',force_remount=True)

df_billboard.to_csv('/content/drive/My Drive/Colab Notebooks/datasets/billboard+features_1.csv')

Mounted at /content/drive
